In [19]:
import torch

import torch.nn as nn

from torchvision import transforms , datasets

from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights


Step 1: Prepare the Dataset (Food101)

In [20]:
transform = transforms.Compose([

transforms.Resize(256),

transforms.CenterCrop(224),

transforms.ToTensor(),

transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

Step 2: Load EfficientNetB0 + Transfer Learning

Freeze backbone initially

Replace the classifier head

Later: Unfreeze for full fine-tuning

In [21]:
weights = EfficientNet_B0_Weights.DEFAULT

model = efficientnet_b0(weights=weights)


for param in model.parameters():

    param.requires_grad = False

# Modify classifier

num_classes = 10

model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

Step 3: Training & Fine-Tuning

Phase 1: Train with frozen backbone

Phase 2: Unfreeze all and fine-tune

Use AdamW or SGD with cosine LR scheduler (optional)

Step 4: Export to ONNX

In [22]:
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

model = model.to(device)

dummy_input = torch.randn(1, 3, 224, 224).to(device)

torch.onnx.export(model, dummy_input, 'efficientnet_b0.onnx',
                  input_names=['input'], output_names=['output'],
                  opset_version=11)

Step 5: Convert to CoreML (on macOS)